# Lab 5: Building AI Agents

In this lab, you'll build **AI agents** that can use tools, make decisions, and accomplish complex tasks autonomously.

## What is an AI Agent?
An AI agent is an LLM that can:
1. **Reason** about how to accomplish a goal
2. **Use tools** to interact with the world
3. **Iterate** based on results
4. **Complete tasks** autonomously

## What You'll Learn
- Building agents with LangChain
- Creating custom tools
- ReAct agents (Reasoning + Acting)
- Multi-step task execution
- Building a customer service agent

## Prerequisites
```bash
pip install langchain langchain-community
ollama pull llama3.2
```

## 1. Setup

In [ ]:
!pip install langchain langchain-community ollama -q

In [ ]:
from langchain_community.llms import Ollama
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool, tool
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from typing import Optional
import json

# Initialize the LLM
llm = Ollama(model="llama3.2")

# Test it
response = llm.invoke("Say 'Hello, Agent!'")
print(response)

## 2. Creating Tools

Tools are functions that agents can call to interact with the world.

In [ ]:
# Simple tool using decorator
@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression. Use this for any math calculations.
    Input should be a valid Python math expression like '2 + 2' or '10 * 5'."""
    try:
        # Safe evaluation of math expressions
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def get_current_time() -> str:
    """Get the current date and time. Use this when asked about the current time or date."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def search_knowledge_base(query: str) -> str:
    """Search the company knowledge base for information.
    Use this to look up product information, policies, or FAQs."""
    # Simulated knowledge base
    knowledge = {
        "return policy": "Items can be returned within 30 days with receipt for full refund.",
        "shipping": "Free shipping on orders over $50. Standard delivery takes 3-5 business days.",
        "warranty": "All products come with a 1-year manufacturer warranty.",
        "hours": "Customer service is available Monday-Friday, 9 AM - 5 PM EST.",
        "contact": "Email: support@example.com, Phone: 1-800-EXAMPLE"
    }
    
    query_lower = query.lower()
    for key, value in knowledge.items():
        if key in query_lower:
            return value
    return "I couldn't find specific information about that. Please contact support for help."

# List our tools
tools = [calculator, get_current_time, search_knowledge_base]
print("Available tools:")
for t in tools:
    print(f"  - {t.name}: {t.description[:60]}...")

## 3. Building a ReAct Agent

ReAct (Reasoning + Acting) agents think step-by-step and use tools to solve problems.

In [ ]:
# ReAct prompt template
react_prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")

# Create the agent
agent = create_react_agent(llm, tools, react_prompt)

# Create the executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Show reasoning
    handle_parsing_errors=True,
    max_iterations=5
)

print("Agent ready!")

In [ ]:
# Test the agent with a simple question
result = agent_executor.invoke({"input": "What is 25 * 17?"})
print("\nFinal Answer:", result["output"])

In [ ]:
# Test with knowledge base
result = agent_executor.invoke({"input": "What is your return policy?"})
print("\nFinal Answer:", result["output"])

In [ ]:
# Multi-step question
result = agent_executor.invoke({
    "input": "If I buy 3 items at $29.99 each, what's my total? And do I get free shipping?"
})
print("\nFinal Answer:", result["output"])

## 4. Building a Customer Service Agent

In [ ]:
# More sophisticated tools for customer service

# Simulated database
orders_db = {
    "ORD-001": {"status": "shipped", "tracking": "1Z999AA10123456784", "items": ["Widget A", "Widget B"], "total": 59.99},
    "ORD-002": {"status": "processing", "tracking": None, "items": ["Gadget X"], "total": 129.99},
    "ORD-003": {"status": "delivered", "tracking": "1Z999AA10123456785", "items": ["Super Gizmo"], "total": 249.99},
}

customers_db = {
    "CUST-100": {"name": "Alice Smith", "email": "alice@example.com", "orders": ["ORD-001", "ORD-003"]},
    "CUST-101": {"name": "Bob Jones", "email": "bob@example.com", "orders": ["ORD-002"]},
}

@tool
def lookup_order(order_id: str) -> str:
    """Look up order details by order ID. Use this when customer asks about their order status.
    Input should be an order ID like 'ORD-001'."""
    order = orders_db.get(order_id.upper())
    if order:
        return json.dumps(order, indent=2)
    return f"Order {order_id} not found."

@tool
def lookup_customer(customer_id: str) -> str:
    """Look up customer information by customer ID.
    Input should be a customer ID like 'CUST-100'."""
    customer = customers_db.get(customer_id.upper())
    if customer:
        return json.dumps(customer, indent=2)
    return f"Customer {customer_id} not found."

@tool
def initiate_return(order_id: str) -> str:
    """Initiate a return for an order. Use when customer wants to return an item.
    Input should be the order ID to return."""
    order = orders_db.get(order_id.upper())
    if order:
        if order["status"] == "delivered":
            return f"Return initiated for {order_id}. Return label will be emailed within 24 hours."
        else:
            return f"Cannot initiate return - order {order_id} status is '{order['status']}'. Only delivered orders can be returned."
    return f"Order {order_id} not found."

@tool
def escalate_to_human(reason: str) -> str:
    """Escalate the conversation to a human agent. Use this when the customer's issue cannot be resolved automatically.
    Input should be a brief description of why escalation is needed."""
    return f"Escalating to human agent. Reason: {reason}. A representative will contact you within 2 hours."

# Customer service tools
cs_tools = [lookup_order, lookup_customer, initiate_return, escalate_to_human, search_knowledge_base]

In [ ]:
# Customer service agent prompt
cs_prompt = PromptTemplate.from_template("""
You are a helpful customer service agent for TechCorp. Be friendly, professional, and helpful.

You have access to the following tools:

{tools}

Use the following format:

Question: the customer's question or request
Thought: think about how to help the customer
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I can now respond to the customer
Final Answer: your helpful response to the customer

Remember:
- Always be polite and helpful
- Look up information before making claims
- If you can't help, offer to escalate

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")

# Create customer service agent
cs_agent = create_react_agent(llm, cs_tools, cs_prompt)
cs_executor = AgentExecutor(
    agent=cs_agent,
    tools=cs_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("Customer service agent ready!")

In [ ]:
# Test customer service scenarios

# Scenario 1: Order status
print("=" * 50)
print("Scenario 1: Order Status Inquiry")
print("=" * 50)
result = cs_executor.invoke({"input": "Hi, I'd like to check on my order ORD-001"})
print("\nAgent Response:", result["output"])

In [ ]:
# Scenario 2: Return request
print("=" * 50)
print("Scenario 2: Return Request")
print("=" * 50)
result = cs_executor.invoke({"input": "I want to return order ORD-003, it's not what I expected"})
print("\nAgent Response:", result["output"])

In [ ]:
# Scenario 3: Policy question
print("=" * 50)
print("Scenario 3: Policy Question")
print("=" * 50)
result = cs_executor.invoke({"input": "Do you offer free shipping? And what's the warranty on your products?"})
print("\nAgent Response:", result["output"])

## 5. Agent with Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

# Create memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Modified prompt with memory
memory_prompt = PromptTemplate.from_template("""
You are a helpful assistant with access to tools.

Previous conversation:
{chat_history}

Tools available:
{tools}

Format:
Question: the input question
Thought: think about what to do
Action: one of [{tool_names}]
Action Input: the input
Observation: the result
Thought: I know the answer
Final Answer: the answer

Question: {input}
Thought: {agent_scratchpad}
""")

# This is simplified - full memory integration requires more setup
print("Memory-enabled agent concept demonstrated")

## 6. Simple Agent Without LangChain

In [ ]:
# You can also build simple agents with just Ollama
import ollama

def simple_agent(user_input: str, max_steps: int = 5) -> str:
    """A simple agent that can use tools."""
    
    # Define available tools
    tool_definitions = """
    Available tools:
    1. calculator(expression) - Evaluates math expressions
    2. search(query) - Searches the knowledge base
    3. done(answer) - Returns final answer
    """
    
    # Simple tool implementations
    def run_tool(tool_name: str, tool_input: str) -> str:
        if tool_name == "calculator":
            try:
                return str(eval(tool_input, {"__builtins__": {}}, {}))
            except:
                return "Error evaluating expression"
        elif tool_name == "search":
            kb = {"shipping": "Free over $50", "returns": "30 day policy"}
            for k, v in kb.items():
                if k in tool_input.lower():
                    return v
            return "Not found"
        return "Unknown tool"
    
    context = f"User: {user_input}\n"
    
    for step in range(max_steps):
        prompt = f"""
        {tool_definitions}
        
        {context}
        
        What tool should be used next? Respond with:
        TOOL: tool_name
        INPUT: tool_input
        
        Or if you have the answer:
        TOOL: done
        INPUT: your final answer
        """
        
        response = ollama.chat(
            model='llama3.2',
            messages=[{'role': 'user', 'content': prompt}]
        )['message']['content']
        
        # Parse response
        lines = response.strip().split('\n')
        tool_name = None
        tool_input = None
        
        for line in lines:
            if line.startswith('TOOL:'):
                tool_name = line.replace('TOOL:', '').strip().lower()
            elif line.startswith('INPUT:'):
                tool_input = line.replace('INPUT:', '').strip()
        
        if tool_name == 'done':
            return tool_input
        
        if tool_name and tool_input:
            result = run_tool(tool_name, tool_input)
            context += f"\nUsed {tool_name}({tool_input}) -> {result}"
    
    return "Could not complete task"

# Test simple agent
print("Simple Agent Test:")
result = simple_agent("What is 15 * 23?")
print(f"Result: {result}")

## Summary

In this lab, you learned how to:
- Create custom tools for agents
- Build ReAct agents with LangChain
- Handle multi-step reasoning
- Create a customer service agent
- Implement simple agents from scratch

**Key takeaways:**
- Agents combine LLMs with tools for autonomous task completion
- ReAct pattern: Reason → Act → Observe → Repeat
- Good tool descriptions are crucial for agent success
- Everything runs locally with Ollama!

**Ideas for extending:**
- Add more tools (web search, file operations, APIs)
- Build a coding assistant agent
- Create multi-agent systems
- Add persistent memory

## Congratulations!

You've completed the Open Source AI Workshop. You now have the skills to:
- Generate text with local LLMs
- Build RAG systems
- Fine-tune models
- Generate and analyze images
- Create AI agents

All running locally, completely free!